In [4]:
import aerospike
import pickle
from datetime import datetime
from collections import Counter
import pymongo
import time
import bson
from tqdm import tqdm
username = "tagger-admin"
password = "tvaiadmin"






db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [ ]:
import boto3
import sys

bucket = 'entity-detector'
prefix = 'comprehend-custom-entity-g1'
comprehend = boto3.client('comprehend', region_name='us-east-2')

def upload_to_s3(s3path, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = s3path
    s3.Bucket(bucket).put_object(Key=key, Body=data)

In [25]:
import boto3

client = boto3.client('comprehend')

In [26]:
import re

In [29]:
db = db_client["tweets_pipeline_v2"].find({"$or": [{"status": "pipelined"},
                                                    {"status": "reserved"},
                                                   ], 
                                      "aws_answer": {"$exists": False}})
for i in tqdm(db, total = db.count()):
    text = re.sub("\n"," ", i['extracted_text'])
    entitites = client.detect_entities(Text=text, LanguageCode='en')["Entities"]
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(str(i['_id']))},
                                            {"$set":{"aws_answer": entitites}
                                            })

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """
100%|██████████| 9566/9566 [16:06<00:00,  9.89it/s]
